# Introducción

En el presente notebook, buscaremos reproducir los resultados obtenidos en el paper de [DELIGHT](https://arxiv.org/pdf/2208.04310).

Para esto, trabajaremos adaptando el modelo propuesto a PyTorch, y mediremos su rendimiento bajo las métricas propuestas por el paper.

# Métricas

Evaluaremos el desempeño de la red mediante 6 métricas sobre el conjunto de test

- $$ RMSE = \sqrt{MSE} = \sqrt{\frac{1}{N} \sum_{i=1}^{N} (Y_i - \hat{Y_i})^2} $$
- $$ Mean Deviaton = \frac{1}{N} \sum_{i=1}^{N} \lVert Y_i - \hat{Y_i} \rVert $$
- $$ Median Deviaton = mediana(\lVert Y_i - \hat{Y_i} \rVert) $$
- $$ Mode Deviaton = moda(\lVert Y_i - \hat{Y_i} \rVert) $$

Donde:
- $N$: Es el tamaño total de los datos del conjunto de test.
- $Y_i$: Es un vector 2d que representa la posición real de la galaxia host. $Y_i = (x_i, y_i)$
- $\hat{Y_i }$: Es un vector 2d que representa la posición predicha de la galaxia host. $\hat{Y_i} = (\hat{x_i}, \hat{y_i})$

# Baseline

Los valores obtenidos para cada métrica en el paper son:
- RMSE: 1.836 ± 0.05100
- Mean Deviation: 0.783 ± 0.00900
- Median Deviation: 0.468 ± 0.00800
- Mode Deviation: 0.427 ± 0.05100


In [1]:
%load_ext autoreload
%load_ext tensorboard
%autoreload 2

In [2]:
import numpy as np
import numpy.typing as npt
from typing import Callable
from scipy import stats  # type: ignore
from sklearn.utils import resample  # type: ignore

StatisticFunction = Callable[[npt.NDArray[np.float32]], float]


def bootstrap_statistic(
    data: npt.NDArray[np.float32],
    statistic: StatisticFunction,
    n_iterations: int = 1000,
) -> float:
    stats = np.zeros(n_iterations)
    for i in range(n_iterations):
        sample: npt.NDArray[np.float32] = resample(data)  # type: ignore
        stats[i] = statistic(sample)
    return np.std(stats).item()


def rmse(
    y_true: npt.NDArray[np.float32], y_pred: npt.NDArray[np.float32]
) -> tuple[float, float]:
    has_shape_2 = len(y_true.shape) == len(y_pred.shape) == 2
    are_points = y_true.shape[1] == y_pred.shape[1] == 2
    assert (
        has_shape_2 and are_points
    ), f"Expected vectors of dim (N, 2): y_true={y_true.shape} y_pred={y_pred.shape}"

    sum_distance_squared: npt.NDArray[np.float32] = np.sum(
        (y_true - y_pred) ** 2, axis=1
    )
    value = np.sqrt(np.mean(sum_distance_squared))  # type: ignore
    assert isinstance(value, float), f"Expected float result: {value}"
    return value, bootstrap_statistic(
        sum_distance_squared, lambda x: np.sqrt(np.mean(x))
    )


def mean_deviation(
    y_true: npt.NDArray[np.float32], y_pred: npt.NDArray[np.float32]
) -> tuple[float, float]:
    has_shape_2 = len(y_true.shape) == len(y_pred.shape) == 2
    are_points = y_true.shape[1] == y_pred.shape[1] == 2
    assert (
        has_shape_2 and are_points
    ), f"Expected vectors of dim (N, 2): y_true={y_true.shape} y_pred={y_pred.shape}"

    deviation: npt.NDArray[np.float32] = np.linalg.norm(y_true - y_pred, axis=1)  # type: ignore
    return np.mean(deviation).item(), bootstrap_statistic(deviation, np.mean)


def median_deviation(
    y_true: npt.NDArray[np.float32], y_pred: npt.NDArray[np.float32]
) -> tuple[float, float]:
    has_shape_2 = len(y_true.shape) == len(y_pred.shape) == 2
    are_points = y_true.shape[1] == y_pred.shape[1] == 2
    assert (
        has_shape_2 and are_points
    ), f"Expected vectors of dim (N, 2): y_true={y_true.shape} y_pred={y_pred.shape}"

    deviation: npt.NDArray[np.float32] = np.linalg.norm(y_true - y_pred, axis=1)  # type: ignore
    return np.median(deviation).item(), bootstrap_statistic(deviation, np.median)


def mode_deviation(
    y_true: npt.NDArray[np.float32], y_pred: npt.NDArray[np.float32]
) -> tuple[float, float]:
    has_shape_2 = len(y_true.shape) == len(y_pred.shape) == 2
    are_points = y_true.shape[1] == y_pred.shape[1] == 2
    assert (
        has_shape_2 and are_points
    ), f"Expected vectors of dim (N, 2): y_true={y_true.shape} y_pred={y_pred.shape}"

    deviation: npt.NDArray[np.float32] = np.linalg.norm(y_true - y_pred, axis=1)  # type: ignore
    mode = stats.mode(deviation, axis=None).mode  # type: ignore
    return mode, bootstrap_statistic(
        deviation,
        lambda x: stats.mode(x).mode,  # type: ignore
    )

In [3]:
import math
from collections import OrderedDict
from typing import TypedDict
from functools import reduce

import torch


class DelightCnnParameters(TypedDict):
    nconv1: int
    nconv2: int
    nconv3: int
    ndense: int
    levels: int
    dropout: float
    rot: bool
    flip: bool


class RotationAndFlipLayer(torch.nn.Module):
    def __init__(self, rot: bool = True, flip: bool = True):
        super().__init__()  # type: ignore
        self.rot = rot
        self.flip = flip
        self.n_transforms = (int(flip) + 1) * (3 * int(rot) + 1)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        stacked = reduce(lambda x, y: x * y, x.shape[:-3], 1)

        if self.rot is False and self.flip is False:
            x = x.reshape(stacked, x.shape[-3], x.shape[-2], x.shape[-1])
            return x

        w_dim = len(x.shape) - 2
        h_dim = len(x.shape) - 1
        transforms: tuple[torch.Tensor, ...]

        if self.rot is False:
            flipped = x.flip(dims=(h_dim,))
            transforms = (x, flipped)

        elif self.flip is False:
            rot90 = x.rot90(k=1, dims=(w_dim, h_dim))
            rot180 = x.rot90(k=2, dims=(w_dim, h_dim))
            rot270 = x.rot90(k=3, dims=(w_dim, h_dim))
            transforms = (x, rot90, rot180, rot270)

        else:
            rot90 = x.rot90(k=1, dims=(w_dim, h_dim))
            rot180 = x.rot90(k=2, dims=(w_dim, h_dim))
            rot270 = x.rot90(k=3, dims=(w_dim, h_dim))
            flipped = x.flip(dims=(h_dim,))
            flipped_rot90 = flipped.rot90(k=1, dims=(w_dim, h_dim))
            flipped_rot180 = flipped.rot90(k=2, dims=(w_dim, h_dim))
            flipped_rot270 = flipped.rot90(k=3, dims=(w_dim, h_dim))
            transforms = (
                x,
                rot90,
                rot180,
                rot270,
                flipped,
                flipped_rot90,
                flipped_rot180,
                flipped_rot270,
            )

        x = torch.cat(transforms, dim=1)
        return x.reshape(
            stacked * self.n_transforms, x.shape[-3], x.shape[-2], x.shape[-1]
        )


class DelightCnn(torch.nn.Module):
    def __init__(self, options: DelightCnnParameters):
        super().__init__()  # type: ignore
        bottleneck: OrderedDict[str, torch.nn.Module] = OrderedDict(
            [
                ("conv1", torch.nn.Conv2d(1, options["nconv1"], 3)),
                ("relu1", torch.nn.ReLU()),
                ("mp1", torch.nn.MaxPool2d(2)),
                ("conv2", torch.nn.Conv2d(options["nconv1"], options["nconv2"], 3)),
                ("relu2", torch.nn.ReLU()),
                ("mp2", torch.nn.MaxPool2d(2)),
                ("conv3", torch.nn.Conv2d(options["nconv2"], options["nconv3"], 3)),
                ("relu3", torch.nn.ReLU()),
                ("flatten", torch.nn.Flatten()),
            ]
        )
        linear_in = self._compute_dense_features(
            levels=options["levels"], bottleneck=bottleneck
        )
        self.fc1 = torch.nn.Linear(
            in_features=linear_in, out_features=options["ndense"]
        )
        self.tanh = torch.nn.Tanh()
        self.dropout = torch.nn.Dropout(p=options["dropout"])
        self.fc2 = torch.nn.Linear(in_features=options["ndense"], out_features=2)
        self.rot_and_flip = RotationAndFlipLayer(
            rot=options["rot"], flip=options["flip"]
        )
        self.bottleneck = torch.nn.Sequential(bottleneck)

    def _compute_dense_features(
        self,
        *,
        bottleneck: OrderedDict[str, torch.nn.Module],
        levels: int,
    ) -> int:
        w = 30
        h = 30
        conv_out = 0
        for layer in bottleneck.values():
            k: int
            if isinstance(layer, torch.nn.Conv2d):
                k = layer.kernel_size[0]
                w = w - k + 1
                h = h - k + 1
                conv_out = layer.out_channels
            if isinstance(layer, torch.nn.MaxPool2d):
                k = layer.kernel_size  # type: ignore
                w = math.floor((w - k) / 2 + 1)
                h = math.floor((h - k) / 2 + 1)

        return w * h * conv_out * levels

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        batch = x.shape[0]  # TODO: Remove batch dependency

        # Apply flips and rotations over level (L) dimension
        x = self.rot_and_flip(x)

        # Bottleneck
        x = self.bottleneck(x)

        # Undo transformations
        x = x.reshape(batch, self.rot_and_flip.n_transforms, -1)

        # Linear
        x = self.fc1(x)
        x = self.tanh(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x.reshape(batch, self.rot_and_flip.n_transforms * 2)

    def derotate(self, y_pred: torch.Tensor) -> npt.NDArray[np.float32]:
        y_pred_numpy: npt.NDArray[np.float32] = y_pred.cpu().numpy()
        return (
            np.dstack(
                [
                    y_pred_numpy.reshape((y_pred_numpy.shape[0], 8, 2))[:, 0],
                    y_pred_numpy.reshape((y_pred_numpy.shape[0], 8, 2))[:, 1, ::-1]
                    * [1, -1],
                    y_pred_numpy.reshape((y_pred_numpy.shape[0], 8, 2))[:, 2, :]
                    * [-1, -1],
                    y_pred_numpy.reshape((y_pred_numpy.shape[0], 8, 2))[:, 3, ::-1]
                    * [-1, 1],
                    y_pred_numpy.reshape((y_pred_numpy.shape[0], 8, 2))[:, 4, :]
                    * [1, -1],
                    y_pred_numpy.reshape((y_pred_numpy.shape[0], 8, 2))[:, 5, ::-1],
                    y_pred_numpy.reshape((y_pred_numpy.shape[0], 8, 2))[:, 6, :]
                    * [-1, 1],
                    y_pred_numpy.reshape((y_pred_numpy.shape[0], 8, 2))[:, 7, ::-1]
                    * [-1, -1],
                ]
            )
            .reshape((y_pred_numpy.shape[0], 2, 8))
            .swapaxes(1, 2)
        )

In [4]:
import os
from dataclasses import dataclass
from enum import Enum
from typing import Any, cast

import numpy as np
import tensorflow as tf
import tensorflow.experimental.numpy as tnp  # type: ignore
import torch
from torch.utils.data import Dataset


class DelightDatasetType(Enum):
    TRAIN = "TRAIN"
    TEST = "TEST"
    VALIDATION = "VALIDATION"


@dataclass
class DelightDatasetOptions:
    source: str
    n_levels: int
    fold: int
    mask: bool
    object: bool
    rot: bool
    flip: bool

    def get_filenames(self, datatype: DelightDatasetType) -> tuple[str, str]:
        if datatype == DelightDatasetType.TRAIN:
            x = "X_train_nlevels%i_fold%i_mask%s_objects%s.npy" % (
                self.n_levels,
                self.fold,
                self.mask,
                self.object,
            )
            y = "y_train_nlevels%i_fold%i_mask%s_objects%s.npy" % (
                self.n_levels,
                self.fold,
                self.mask,
                self.object,
            )
        elif datatype == DelightDatasetType.TEST:
            x = "X_test_nlevels%i_mask%s_objects%s.npy" % (
                self.n_levels,
                self.mask,
                self.object,
            )
            y = "y_test_nlevels%i_mask%s_objects%s.npy" % (
                self.n_levels,
                self.mask,
                self.object,
            )
        else:
            x = "X_val_nlevels%i_fold%i_mask%s_objects%s.npy" % (
                self.n_levels,
                self.fold,
                self.mask,
                self.object,
            )
            y = "y_val_nlevels%i_fold%i_mask%s_objects%s.npy" % (
                self.n_levels,
                self.fold,
                self.mask,
                self.object,
            )

        return x, y


class DelightDataset(Dataset[tuple[torch.Tensor, torch.Tensor]]):
    def __init__(self, options: DelightDatasetOptions, datatype: DelightDatasetType):
        X_path, y_path = options.get_filenames(datatype)
        self.X = torch.Tensor(np.load(os.path.join(options.source, X_path))).permute(
            0, 3, 1, 2
        )

        y = np.load(os.path.join(options.source, y_path))
        self.y = (
            self.transform(
                y,
                options.rot,
                options.flip,
            )
            if datatype != DelightDatasetType.TEST
            else torch.from_numpy(y)  # type: ignore
        )
        self.y_raw = np.load(os.path.join(options.source, y_path))

    @staticmethod
    def transform(
        y: np.ndarray[Any, np.dtype[np.float32]], rot: bool, flip: bool
    ) -> torch.Tensor:
        transformed: tuple[np.ndarray[Any, np.dtype[np.float32]], ...]

        if rot is False and flip is False:
            return torch.Tensor(y)

        yflip = cast(np.ndarray[Any, np.dtype[np.float32]], [1, -1] * y)
        if rot is False:
            transformed = (y, yflip)

        y90 = cast(np.ndarray[Any, np.dtype[np.float32]], [-1, 1] * y[:, ::-1])
        y180 = cast(np.ndarray[Any, np.dtype[np.float32]], [-1, 1] * y90[:, ::-1])
        y270 = cast(np.ndarray[Any, np.dtype[np.float32]], [-1, 1] * y180[:, ::-1])
        yflip90 = cast(np.ndarray[Any, np.dtype[np.float32]], [-1, 1] * yflip[:, ::-1])
        yflip180 = cast(
            np.ndarray[Any, np.dtype[np.float32]], [-1, 1] * yflip90[:, ::-1]
        )
        yflip270 = cast(
            np.ndarray[Any, np.dtype[np.float32]], [-1, 1] * yflip180[:, ::-1]
        )

        if flip is False:
            transformed = (y, y90, y180, y270)
        else:
            transformed = (y, y90, y180, y270, yflip, yflip90, yflip180, yflip270)

        return torch.Tensor(np.concatenate(transformed, axis=1))

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx: int):
        x = self.X[idx]
        y = self.y[idx]

        if len(x.shape) == 3:  # has no channel information
            levels, width, height = x.shape
            x = x.reshape(levels, 1, width, height)  # asume 1 channel information
        return x, y

    def to_tf_dataset(self) -> tuple[tf.Tensor, tf.Tensor]:
        X = cast(np.ndarray[Any, np.dtype[np.float32]], self.X.numpy())
        y = cast(np.ndarray[Any, np.dtype[np.float32]], self.y.numpy())

        return tnp.copy(X.transpose((0, 2, 3, 1))), tnp.copy(y)  # type: ignore

2024-06-14 01:38:48.649839: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-14 01:38:48.649901: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-14 01:38:48.649933: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-14 01:38:48.661629: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-14 01:38:50.112906: W tensorflow/compiler/

In [5]:
import tempfile
from typing import TypedDict
import datetime

import torch
from ray import train
from ray.train import Checkpoint
from torch.utils.data import DataLoader
from torch.utils.tensorboard.writer import SummaryWriter

class HyperParameters(TypedDict):
    lr: float
    batch_size: int | float
    nconv1: int | float
    nconv2: int | float
    nconv3: int | float
    ndense: int | float
    dropout: float
    epochs: int


class EvaluationResult(TypedDict):
    rmse: tuple[float, float]
    mean_deviation: tuple[float, float]
    median_deviation: tuple[float, float]
    mode_deviation: tuple[float, float]


def _get_value_from_parameter(parameter: int | float, base: int = 2) -> int:
    return int(base**parameter) if isinstance(parameter, float) else parameter


def get_delight_cnn_parameters(
    params: HyperParameters, options: DelightDatasetOptions
) -> DelightCnnParameters:
    return {
        "nconv1": _get_value_from_parameter(params["nconv1"]),
        "nconv2": _get_value_from_parameter(params["nconv2"]),
        "nconv3": _get_value_from_parameter(params["nconv3"]),
        "ndense": _get_value_from_parameter(params["ndense"]),
        "levels": options.n_levels,
        "dropout": params["dropout"],
        "rot": options.rot,
        "flip": options.flip,
    }


def _train_one_epoch(
    *,
    device: str,
    train_dl: DataLoader[tuple[torch.Tensor, torch.Tensor]],
    optimizer: torch.optim.Optimizer,
    model: DelightCnn,
    criterion: torch.nn.MSELoss,
    writer: SummaryWriter,
    is_ray: bool,
    epoch: int
):
    running_loss = 0.
    inputs: torch.Tensor
    positions: torch.Tensor
    outputs: torch.Tensor
    loss: torch.Tensor

    model.train()
    for i, (inputs, positions) in enumerate(train_dl):
        inputs, positions = inputs.to(device), positions.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)

        loss = criterion(outputs, positions)
        loss.backward()  # type: ignore

        optimizer.step()
        loss_value = loss.item() 

        if is_ray is False:
            t = epoch * len(train_dl) + i # type: ignore
            writer.add_scalar("[MSE Loss]: Train", loss_value, t)  # type: ignore

        running_loss += loss_value * inputs.size(0)

    return running_loss / len(train_dl.dataset)  # type: ignore


def _validate_train(
    *,
    device: str,
    val_dl: DataLoader[tuple[torch.Tensor, torch.Tensor]],
    model: DelightCnn,
    criterion: torch.nn.MSELoss,
):
    running_loss = 0.
    data: tuple[torch.Tensor, torch.Tensor]
    outputs: torch.Tensor
    loss: torch.Tensor

    model.eval()
    with torch.no_grad():
        for _, data in enumerate(val_dl):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)

            loss = criterion(outputs, labels)
            running_loss += loss.item() * labels.size(0)

    return running_loss / len(val_dl.dataset) # type: ignore


def _train(
    *,
    start_epoch: int,
    num_epochs: int,
    device: str,
    train_dl: DataLoader[tuple[torch.Tensor, torch.Tensor]],
    val_dl: DataLoader[tuple[torch.Tensor, torch.Tensor]],
    optimizer: torch.optim.Optimizer,
    model: DelightCnn,
    criterion: torch.nn.MSELoss,
    is_ray: bool = False,
):
    model.to(device)
    writer = SummaryWriter(comment=datetime.datetime.now().strftime("%Y-%m-%dT%H-%M-%SZ"))
    for epoch in range(start_epoch, num_epochs):
        train_loss = _train_one_epoch(
            device=device,
            train_dl=train_dl,
            optimizer=optimizer,
            model=model,
            criterion=criterion,
            is_ray=is_ray,
            writer=writer,
            epoch=epoch
        )

        val_loss = _validate_train(
            device=device, val_dl=val_dl, model=model, criterion=criterion
        )

        print(f"[EPOCH {epoch+1}] train loss = {train_loss} | val_loss = {val_loss}")
        metrics = {"val_loss": val_loss, "train_loss": train_loss}
        
        if is_ray is False:
            writer.add_scalars("[MSE Loss]: Train / Validation", metrics, epoch)  # type: ignore
            continue

        with tempfile.TemporaryDirectory() as tempdir:
            torch.save(  # type: ignore
                {
                    "epoch": epoch,
                    "net_state_dict": model.state_dict(),
                    "optimizer_state_dict": optimizer.state_dict(),
                },
                os.path.join(tempdir, "checkpoint.pt"),
            )
            train.report(metrics=metrics, checkpoint=Checkpoint.from_directory(tempdir))  # type: ignore

    writer.close()

def train_delight_cnn_model(
    params: HyperParameters, options: DelightDatasetOptions
) -> DelightCnn:
    device = "cpu" if torch.cuda.is_available() is False else "cuda"
    batch_size = _get_value_from_parameter(params["batch_size"])
    net_options = get_delight_cnn_parameters(params, options)
    net = DelightCnn(net_options)

    criterion = torch.nn.MSELoss()
    optimizer = torch.optim.Adam(net.parameters(), lr=params["lr"], weight_decay=1e-4)
    checkpoint = cast(Checkpoint | None, train.get_checkpoint())  # type: ignore
    start_epoch = 0

    if checkpoint:
        with checkpoint.as_directory() as checkpoint_dir:
            checkpoint_dict = torch.load(os.path.join(checkpoint_dir, "checkpoint.pt"))  # type: ignore
            start_epoch = int(checkpoint_dict["epoch"]) + 1
            net.load_state_dict(checkpoint_dict["net_state_dict"])
            optimizer.load_state_dict(checkpoint_dict["optimizer_state_dict"])

    train_dataset = DelightDataset(options=options, datatype=DelightDatasetType.TRAIN)
    val_dataset = DelightDataset(
        options=options, datatype=DelightDatasetType.VALIDATION
    )
    train_dl = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
    val_dl = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    print(
        "Starting: epochs=%s,batch_size=%s,lr=%s,nconv1=%s,nconv2=%s,nconv3=%s,ndense=%s,dropout=%s"
        % (
            params["epochs"],
            batch_size,
            params["lr"],
            net_options["nconv1"],
            net_options["nconv2"],
            net_options["nconv3"],
            net_options["ndense"],
            net_options["dropout"],
        )
    )

    _train(
        start_epoch=start_epoch,
        num_epochs=params["epochs"],
        device=device,
        train_dl=train_dl,
        val_dl=val_dl,
        optimizer=optimizer,
        model=net,
        criterion=criterion,
        is_ray=checkpoint is not None,
    )

    return net


def evaluate_delight_cnn_model(
    model: DelightCnn, options: DelightDatasetOptions
) -> EvaluationResult:
    device = "cpu" if torch.cuda.is_available() is False else "cuda"
    test_dataset = DelightDataset(options=options, datatype=DelightDatasetType.TEST)
    test_dl = DataLoader(test_dataset, batch_size=16, shuffle=False)

    print("Evaluating model...")
    predictions: list[tuple[float, float]] = []

    model.to(device)
    model.eval()

    inputs: torch.Tensor
    outputs: torch.Tensor
    with torch.no_grad():
        for _, data in enumerate(test_dl):
            inputs, _ = data
            inputs = inputs.to(device)
            outputs = model(inputs)
            derotated = model.derotate(outputs)
            y_hat: npt.NDArray[np.float32] = np.mean(derotated, axis=1)
            predictions.extend(y_hat.tolist())

    y_true: npt.NDArray[np.float32] = test_dataset.y.cpu().numpy()
    y_pred: npt.NDArray[np.float32] = np.array(predictions)

    return {
        "rmse": rmse(y_true, y_pred),
        "mean_deviation": mean_deviation(y_true, y_pred),
        "median_deviation": median_deviation(y_true, y_pred),
        "mode_deviation": mode_deviation(y_true, y_pred),
    }

In [9]:
%tensorboard --logdir=runs

options = DelightDatasetOptions(
    source=os.path.join(os.getcwd(), "data"),
    n_levels=5,
    fold=0,
    mask=False,
    object=True,
    rot=True,
    flip=True,
)

params_paper: HyperParameters = {
    "nconv1": 52,
    "nconv2": 57,
    "nconv3": 41,
    "ndense": 685,
    "dropout": 0.06,
    "epochs": 50,
    "batch_size": 40,
    "lr": 0.0014,
}

params: HyperParameters = {
    "nconv1": 16,
    "nconv2": 32,
    "nconv3": 32,
    "ndense": 128,
    "dropout": 0,
    "epochs": 50,
    "batch_size": 64,
    "lr": 0.0014,
}

model = train_delight_cnn_model(params_paper, options)

Reusing TensorBoard on port 6006 (pid 158132), started 0:05:43 ago. (Use '!kill 158132' to kill it.)

Starting: epochs=50,batch_size=40,lr=0.0014,nconv1=52,nconv2=57,nconv3=41,ndense=685,dropout=0.06
[EPOCH 1] train loss = 252.0790331670234 | val_loss = 219.78536707793148
[EPOCH 2] train loss = 128.32898105003758 | val_loss = 169.39433623988066
[EPOCH 3] train loss = 107.8085772997386 | val_loss = 152.15092055346943
[EPOCH 4] train loss = 95.75713814949381 | val_loss = 107.3790698150711
[EPOCH 5] train loss = 82.04846020827127 | val_loss = 96.13334832362192
[EPOCH 6] train loss = 72.78262585863851 | val_loss = 83.45232794743394
[EPOCH 7] train loss = 65.6467670130891 | val_loss = 76.52644894462541
[EPOCH 8] train loss = 58.91023311485914 | val_loss = 73.56522845189637
[EPOCH 9] train loss = 53.11343988659852 | val_loss = 70.41715534124445
[EPOCH 10] train loss = 48.8168454837948 | val_loss = 67.26101846242328
[EPOCH 11] train loss = 44.83924540351419 | val_loss = 64.83426743919506
[EPOCH 12] train loss = 41.96766791375958 | val_loss = 61.76436128862494
[EPOCH 13] train loss = 38.970078

In [10]:
evaluation = evaluate_delight_cnn_model(model, options)

evaluation

Evaluating model...


{'rmse': (8.29558022797704, 0.6221508076956249),
 'mean_deviation': (3.698574798348921, 0.10675042863577326),
 'median_deviation': (2.009651905374769, 0.02004303353270777),
 'mode_deviation': (2.9544196874954225, 4.743186788967148)}